In [1]:
# Import Dependencies
import json
import mechanicalsoup
import pymongo

# setup mongodb connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# select database
db = client.project2

browser = mechanicalsoup.StatefulBrowser()

# test variables
# search_job = ['data scientist', 'data engineer']
# search_city = ['san diego', 'san francisco']

# production variables
search_job = ['data scientist', 'data engineer', 'data analyst', 'statistician', 'data & analytics manager']
search_city = ['ann arbor', 'atlanta', 'austin', 'boston', 'charlotte', 'chicago', 'cincinnati', 'columbia', 'dallas', 'denver', 'houston', 'jacksonville', 'los angeles', 'louisville', 'miami', 'minneapolis', 'nashville', 'new york', 'newport news', 'phoenix', 'san diego', 'san francisco', 'seattle', 'virginia beach']

# Job Count Scraper Function

In [2]:
def jobCount(job, city):

    try:
        url = "https://www.careerbuilder.com/insights"

        browser.open(url)
        browser.select_form("input")
        browser.select_form('form[action="https://www.careerbuilder.com/jobs"]')
        browser["keywords"] = job
        browser["location"] = city
        response = browser.submit_selected()
        soup = response.soup
        count = soup.find("div", class_="job-results").find("div", class_="count").text.replace(" Jobs", "")\
        .replace("(","").replace(")","")
        count = int((count))
    
        return count
    
    except:
        print(f'Error occurred for ' + job + " in " + city)
        pass

In [3]:
# Run through loops & run function to get count of number of each job per city
result_dict = {}

for job in search_job:
    # new dictionary for city
    city_dict = {}
    
    for city in search_city:
        # load to dictionary with city as the key
        city_dict[city] = jobCount(job, city) #, careerInfo(job, city)
        
    # load to dictionary with job as the key
    result_dict[job] = city_dict

Error occurred for data scientist in newport news
Error occurred for data scientist in virginia beach
Error occurred for statistician in columbia


In [4]:
# load geojson to mongodb
db.job_counts.update_one({}, {'$set': result_dict}, upsert=True)

In [ ]:
# Turn dictionary into JSON file
# with open('job_counts.json', 'w') as fp:
#     json.dump(result_dict, fp)